<center><H1>Modelo neuronal denso para la predicción de la demanda de energía eléctrica</H1><center>

<center><img src="https://www.gstatic.com/devrel-devsite/prod/ve2848ad92313fddfcd40baeb58a2f663fe2fd55c371a714a6bb3e329e2b15223/tensorflow/images/lockup.svg"  height="80px" style="padding-bottom:5px;"  /></center>

<center><H2>Julio Waissman Vilanova</H2>

<table align="center">
      <td align="center"><a target="_blank" href="https://www.unison.mx">
            <img src="https://www.unison.mx/wp-content/themes/awaken/images/logo.png"  height="70px" style="padding-bottom:5px;"  /></a></td>  
      <td align="center"><a target="_blank" href="https://www.gob.mx/cenace">
            <img src="https://universidad.cenace.gob.mx/pluginfile.php/244/block_html/content/CENACE-logo-completo.png" width="300" style="padding-bottom:5px;" /></a></td>
      <td align="center"><a target="_blank" href="https://colab.research.google.com/github/juliowaissman/rn-cenace/blob/main/RedDensa_datos_GRNO.ipynb">
            <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a></td>

</table>

In [ ]:
!pip install plotly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import plotly.express as px
import plotly.graph_objects as go

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15,7)

## 1. Introducción

Vamos ahora a ver como preparar los datos para poderlos usar con una red densa. Los métodos que vamos a utilizar son aplicables tambien para generar series de tiempo de entrenamiento para redes recurrentes y para una combinación de CNN con LSTM.

En esta libreta, al ser tipo taller, vamos a tener pocos comentarios, pero los vamos a ir completando poco a poco


## 2. Generando datos de aprendizaje

Vamos aprobar algunas funciones de Keras para poder preprocesar series de tiempo, para tener una idea clara, nos vamos ainventar una serie de tiempo muy fácil de visualizar y ver como funcionan las herramientas.

Así que vamos a hacer una serie de tiempo de una sola variable (pirata, realizada solo con la secuencia de números desde 0 hasta 10 mil

In [ ]:
serie_pirata = np.arange(10_000, dtype=np.float32)

serie_pirata

Y ahora usaremos [`timeseries_dataset_from_array`](https://keras.io/api/preprocessing/timeseries/) para generar un conjunto de datos de aprendizaje de una sola serie (i.e. la demanda de energía)

In [ ]:
def genera_serie(serie, horizonte):
    """Genera una dataset para entrenar una red neuronal de una serie de tiempo"""
    
    input_data = serie[:-horizonte]
    targets = serie[horizonte:]

    ## Esta es la función para generar series
    ## Vamos a probar direrentes parámetros para entender bien
    dataset = keras.preprocessing.timeseries_dataset_from_array(
        input_data, targets, 
        sequence_length=horizonte, 
        shuffle=False,
        sampling_rate=1,
        sequence_stride=1,
        batch_size=10
    )
    return dataset


dataset_pirata = genera_serie(serie_pirata, 10)

for i, batch in enumerate(dataset_pirata):
    x, y = batch
    print(f"x es del tipo {type(x)}")
    print(f"y es del tipo {type(y)}")
    print(f"Minibarch {i+1}")
    print(f"x = \n{x}")
    print(f"y = {y}")
    if i > 1:
        break

¿Interesante no? Podemos generar un conjunto para aprender stableciendo cuantos datos anteriores vamos a usar para entrenamiento. Y estos datos ya vienen en forma de un [Dataset](https://www.tensorflow.org/guide/data) que puede ser utilizado por Tensorflow para el entrenamiento.

## 3. Cargando los datos y apartando la serie de tiempo

Vamos entonces a usar los datos que nos proporcionó Jesús sobre la demanda de energñia en la Gerencia Regional Noroeste, y vamos a aprender sobre una serie de tiempo, a ver que pasa.

In [ ]:
url = "https://github.com/juliowaissman/curso-ml-cenace/raw/main/datos/Dataset_GCRNO_05052021.xlsx"

df = pd.read_excel(url, sheet_name='Datos')

df_horario = df.melt(
    id_vars= ['FECHA'],
    value_vars= [f'DEM_GCRNO_H{i}' for i in range(24)],
    var_name="Hora",
    value_name="Demanda"
).replace(
    {f'DEM_GCRNO_H{i}': i for i in range(24)}
)

df_horario.index = df_horario.FECHA + pd.to_timedelta(df_horario.Hora, unit='h')
df_horario.sort_index(inplace=True)
df_horario.drop(columns=['FECHA', 'Hora'], inplace=True)
df_horario = df_horario.asfreq('H', method='pad')

fig = px.line(df_horario, x=df_horario.index, y="Demanda", title='Demanda de energía GRNO')
fig.show()


In [ ]:
df_horario['DeseasonW'] = df_horario.Demanda.diff(7*24)
fig = px.line(df_horario, x=df_horario.index, y="DeseasonW", title='Incremento diario de la demanda de energía GRNO')
fig.show()

df_horario['DeseasonD'] = df_horario.Demanda.diff(24)
fig = px.line(df_horario, x=df_horario.index, y="DeseasonD", title='incremento semanal-diario del Demanda de energía GRNO')
fig.show()

df_horario['Deseason'] = df_horario.Demanda.diff()
fig = px.line(df_horario, x=df_horario.index, y="Deseason", title='incremento semanal-diario del Demanda de energía GRNO')
fig.show()



In [ ]:
serie_train = df_horario.Demanda[df_horario.index.year < 2021].to_numpy()
serie_test = df_horario.Demanda[df_horario.index.year == 2021].to_numpy()

horizonte = 24*7 + 12 #(una semana de pasado)
dataset_train = genera_serie(serie_train, horizonte)
dataset_test = genera_serie(serie_test, horizonte)

print(f"Shape de serie_train = {serie_train.shape}")
print(f"Shape de serie_test = {serie_test.shape}")

## 4. Haciendo y entrenando un modelo

Vamos a hacer un modelo rápido y furioso, a ver si nos sale algo bueno

In [ ]:
def modelo_sin_compil(nombre="gallina_pinta"):
    modelo = keras.models.Sequential(name=nombre)

    #TODO: ajusta un modelo
    modelo.add(layers.Dense(256, activation="relu", input_shape=(horizonte,), name="capa_1"))
    modelo.add(layers.Dense(256, activation="relu", name="capa_2"))
    modelo.add(layers.Dense(256, activation="relu", name="capa_3"))
    modelo.add(layers.Dense(1, activation="linear", name="capa_salida"))
    return modelo

def modelo_compilado():
    model = modelo_sin_compil()
    model.compile(
        optimizer=keras.optimizers.Adam(),
        loss=keras.losses.MeanAbsoluteError(),
        metrics=[
            keras.metrics.MeanAbsolutePercentageError()
        ],
    )
    return model

modelo1 = modelo_compilado()
modelo1.summary()

In [ ]:
num_epochs = 2
print(f"Entrenamiento del modelo en {num_epochs} epochs")

history = modelo1.fit(
    dataset_train,
    epochs=num_epochs,
)

In [ ]:
results = modelo1.evaluate(dataset_test)

print("\n\nMAE en test, MAPE en test:", results)

## 5. Probando que tan bien funciona

Por supuesto que la evaluación es solo para ver si lo que aprendió lo puede usar con los datos de test, pero la prueba está en estimar las 36 horas, y quedarse con las 24. 

Vamos a hacer una función que nos permita hacer este tipo de predicciones y ver que tal nos funciona el modelo ya en la realidad


In [ ]:
df_test = df_horario.Demanda[df_horario.index.year == 2021]

y_real = serie_test[horizonte + 12:]
y_est_l = []

for ini in range(0, serie_test.shape[0] - horizonte - 12, 24):
    x = serie_test[ini:ini+horizonte]
    for inc in range(36):
        y = float(modelo1.predict(x.reshape(1, -1)))
        x = np.append(x[1:], y)
        if inc >= 12:
            y_est_l.append(y)

y_est = np.array(y_est_l)

In [ ]:
df_estimacion = pd.DataFrame({
    "Fecha": df_test.index[horizonte + 12:],
    "Real": y_real,
    "Estimado": y_est
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_estimacion.Fecha, y=df_estimacion.Estimado, name="Estimada"))
fig.add_trace(go.Scatter(x=df_estimacion.Fecha, y=df_estimacion.Real, name="Real"))
fig.update_layout(title="Estimación de la demanda")
fig.show()



No está nada optimizado el código de reconocimiento, pero es más o menos funcional. Con lo que podemos probar diferentes modelos neuronales.